In [1]:
import csv
from collections import defaultdict
import re
import random
import torch
import hydra
import datasets
from omegaconf import OmegaConf
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from pathlib import Path
import sys

sys.path.append("..")

from data import get_task

In [2]:
ckpt_dir = Path("/home/sid/hypernetwork-editor/assets/checkpoints/scone_ft_20240723_185251/step-80")

In [3]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name).to("cuda")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# load checkpoint
model.load_state_dict(torch.load(ckpt_dir / "checkpoint.pt", map_location="cpu")["state"])

/tmp/ipykernel_3621547/3606339382.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_dir / "checkpoint.pt", map_location="cpu")["state

<All keys matched successfully>

In [4]:
tokenizer.pad_token_id = tokenizer.eos_token_id

In [5]:
def generate_text(input_ids: torch.Tensor, max_new_tokens=100):
    # Load pre-trained model and tokenizer
    input_ids = input_ids.unsqueeze(0).cuda()

    # Generate text
    output = model.generate(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        temperature=1e-5,
    )
    
    # Decode the generated text
    generated_text = tokenizer.decode(
        output.squeeze(), skip_special_tokens=False
    )

    return generated_text

In [6]:
config = OmegaConf.load(ckpt_dir / "config.yaml")

In [7]:
dataset = get_task(config, config.task.name, "val")

In [8]:
example = random.choice(dataset)

In [9]:
prompt, label = example["instruction"], example["target"]
ids = torch.tensor(example["input_ids"])
labels = torch.tensor(example["labels"])
mask = torch.tensor(example["attention_mask"])
ids = ids[torch.logical_and(labels < 0, mask > 0)]
labels = labels[labels >=0 ]

In [10]:
input_text = tokenizer.decode(ids)
trailing_instr = input_text.split("\n")[-1]
tokenized_trailing_instr = tokenizer.encode(trailing_instr + "\n")
tokenized_trailing_instr.insert(0, tokenizer.bos_token_id)

In [17]:
print(input_text)

<|endoftext|>pour out one part of the second yellow beaker


In [18]:
resp = generate_text(ids)

/home/sid/miniconda3/envs/editor/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [19]:
print(resp.replace(tokenizer.decode(ids), "").strip())

the first beaker has 3 purple, the second beaker has 2 red, the third beaker has 1 green, the fourth beaker has 1 red, the fifth beaker has 1 green, the sixth beaker has 1 red<|endoftext|>


In [20]:
print(label)

the first beaker has 1 yellow, the second beaker has 1 red, the third beaker has 1 red, the fourth beaker has 2 green, the fifth beaker has 3 green, the sixth beaker has 2 yellow, the seventh beaker has 2 yellow
